# Convolution in 1D

Simple Implementation of convolution in 1D.

Convolution in one dimension works as follows.

![alt_text](https://qph.ec.quoracdn.net/main-qimg-523434af0d21bb0b59454aa9563cc90b-c)

In [1]:
# Importing the dependencies
import tensorflow as tf
import numpy as np

### Making a toy data

We create a simple sequential data, where sequence length is 200 and each value ranges between [-1000, 1000].

The output has three classes.

In [2]:
# toy data params
seq_len = 200 # length of training sequence
examples = 100 # number of examples
val_high = 1000 # maximum value in the sequence
val_low = -1000 # minimum value in sequence
# we select this type of max-min values because we often find sequential data to be
# extremely sparse and-or of large fluctuations in values
n_classes = 3 # number of classes in output

# hyper params
n_epochs = 200

In [3]:
# making input data
data_x = np.random.randint(high = val_high, low = val_low, size = (examples, seq_len, 1))
print('[*]Shape of input data:', data_x.shape[1:])

# making output data
# making output data
data_y = np.zeros(shape = (examples, n_classes))
for i in range(len(data_y)):
    data_y[i][np.random.randint(n_classes)] = 1.0
print('[*]Shape of output data:', data_y.shape[1:])

[*]Shape of input data: (200, 1)
[*]Shape of output data: (3,)


### Making the model

Input --> Convolution_Layer --> Dense_Layer --> Output --> Loss <-- Actual_Output

In [4]:
# Defining the placeholders
x = tf.placeholder(tf.float32, shape = [None, seq_len, 1], name = 'x')
y = tf.placeholder(tf.float32, shape = [None, n_classes], name = 'y')

In [5]:
'''
For convolution API given
tf.nn.conv1d(input, filter, strides, padding, ...)
input = input to the convolution layer --> should be a 3D tensor [batch_size, in_width, in_channels]
filter = weights to the convolution layer --> should be a 3D tensor
strides = An integer defining by how much the filter should move
padding = "SAME" or "VALID"
'''

# Defining the weights a.k.a. filter
# it is of shape --> [sliding_window_size, in_channels, out_channels]
W_conv = tf.Variable(tf.truncated_normal([20, 1, 10]), name = 'W_conv')
# defining bias
b_conv = tf.Variable(tf.truncated_normal([10]), name = 'b_conv')
# doing the convolution
h_conv = tf.nn.conv1d(x, W_conv, stride = 5, padding = 'SAME') + b_conv
# the output of the conv1d is [?, seq_len/stride, out_channel]

# Dense Layer
W_dense = tf.Variable(tf.truncated_normal([40*10, n_classes]), name = 'W_dense')
b_dense = tf.Variable(tf.truncated_normal([n_classes]), name = 'W_dense')
# flattening the output of convolution layer
h_conv_flat = tf.reshape(h_conv, [-1, 40*10])
y_ = tf.matmul(h_conv_flat, W_dense) + b_dense

In [6]:
# Determining the loss function and accuracy function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# Now we train the model
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for e in range(n_epochs):
    feed_dict = {x: data_x, y: data_y}
    _, = sess.run([train_step], feed_dict = feed_dict)
    if (e+1) % 20 == 0:
        acc = sess.run([accuracy], feed_dict = feed_dict)
        print('Epoch {0}, accuracy : {1}'.format(e+1, acc[0]))

Epoch 20, accuracy : 0.38999998569488525
Epoch 40, accuracy : 0.38999998569488525
Epoch 60, accuracy : 0.46000000834465027
Epoch 80, accuracy : 0.46000000834465027
Epoch 100, accuracy : 0.5
Epoch 120, accuracy : 0.550000011920929
Epoch 140, accuracy : 0.5899999737739563
Epoch 160, accuracy : 0.6399999856948853
Epoch 180, accuracy : 0.6899999976158142
Epoch 200, accuracy : 0.6700000166893005


As we can see the accuracy increases